In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Clear_Cell_Carcinoma"
cohort = "GSE94321"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Clear_Cell_Carcinoma"
in_cohort_dir = "../../input/GEO/Kidney_Clear_Cell_Carcinoma/GSE94321"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/GSE94321.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE94321.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/clinical_data/GSE94321.csv"
json_path = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Embryonic signature distinguishes pediatric and adult rhabdoid tumors from other SMARCB1-deficient cancers"
!Series_summary	"We used microarrays to compare gene expression profilings in various SMARCB1-deficient tumors."
!Series_overall_design	"[human mRNA] 16 RT, 16 SD-NRT (8 Epithelioid Sarcomas, 3 Undifferentiated Chordomas and 5 Renal Medullary Carcinomas), 37 SMARCB1 deficient tumors."
Sample Characteristics Dictionary:
{0: ['tissue: SMARCB1 deficient tumor', 'tissue: RT', 'tissue: RMC', 'tissue: ES', 'tissue: UC']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# The sample characteristics dictionary was provided in the previous step
sample_characteristics = {0: ['tissue: SMARCB1 deficient tumor', 'tissue: RT', 'tissue: RMC', 'tissue: ES', 'tissue: UC']}

# Create a DataFrame from the sample characteristics dictionary
clinical_data = pd.DataFrame()
for row_idx, values in sample_characteristics.items():
    for value in values:
        clinical_data.loc[row_idx, value] = value

# 1. Gene Expression Data Availability
# Based on the series overall design mentioning "human mRNA", this dataset likely contains gene expression data
is_gene_available = True

# 2.1 Data Availability
# For trait (Kidney Clear Cell Carcinoma):
# Looking at the sample characteristics, it appears to be RMC (Renal Medullary Carcinoma)
# which is a type of kidney cancer, so row 0 contains relevant information
trait_row = 0

# Age and gender information are not available in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion

def convert_trait(value: str) -> int:
    """Convert trait value to binary (0: no cancer, 1: has cancer)."""
    if value is None or pd.isna(value):
        return None
    
    # Extract the value after colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # RMC (Renal Medullary Carcinoma) is a type of kidney cancer
    if 'RMC' in value:
        return 1
    # Clear Cell Carcinoma - we're looking for Kidney Clear Cell Carcinoma
    elif 'SMARCB1 deficient tumor' in value:
        return 1
    # RT (Rhabdoid Tumor) is not a kidney cancer
    elif 'RT' in value or 'ES' in value or 'UC' in value:
        return 0
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """Convert age value to float."""
    # Age is not available
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender value to binary (0: female, 1: male)."""
    # Gender is not available
    return None

# 3. Save Metadata
# Trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if is_trait_available:
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the selected clinical features to a CSV file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'tissue: SMARCB1 deficient tumor': [1.0], 'tissue: RT': [0.0], 'tissue: RMC': [1.0], 'tissue: ES': [0.0], 'tissue: UC': [0.0]}
Clinical data saved to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/clinical_data/GSE94321.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:
Successfully extracted gene data with 18989 rows
First 20 gene IDs:
Index(['100009676_at', '10000_at', '10001_at', '10002_at', '10003_at',
       '100048912_at', '100049716_at', '10004_at', '10005_at', '10006_at',
       '10007_at', '10008_at', '100093630_at', '10009_at', '1000_at',
       '100101467_at', '100101938_at', '10010_at', '100113407_at', '10011_at'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Analyze the gene identifiers format
# The identifiers like '100009676_at', '10000_at', etc. appear to be probe IDs from a microarray
# These are not standard human gene symbols (which would look like BRCA1, TP53, etc.)
# The "_at" suffix is characteristic of Affymetrix probe IDs that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 1329299 rows

Gene annotation preview (first few rows):
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'ENTREZ_GENE_ID': ['1', '10', '100', '1000', '10000'], 'Description': ['alpha-1-B glycoprotein', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'adenosine deaminase', 'cadherin 2, type 1, N-cadherin (neuronal)', 'v-akt murine thymoma viral oncogene homolog 3 (protein kinase B, gamma)'], 'SPOT_ID': [nan, nan, nan, nan, nan]}

Column names in gene annotation data:
['ID', 'ENTREZ_GENE_ID', 'Description', 'SPOT_ID']

The dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.
Example SPOT_ID format: nan


### Step 6: Gene Identifier Mapping

In [7]:
# Gene Identifier Mapping

# Analyze which columns correspond to gene identifiers and gene symbols
print("Analyzing gene identifier and symbol columns...")

# Look at the gene expression index (probes)
print("Comparing gene expression IDs with annotation IDs...")
expression_id_example = gene_data.index[0]
print(f"Example ID from expression data: {expression_id_example}")

# Check overlap between expression data and annotation data
expression_ids = set(gene_data.index)
annotation_ids = set(gene_annotation['ID'])
common_ids = expression_ids & annotation_ids
print(f"Number of matching probe IDs: {len(common_ids)} out of {len(gene_data.index)}")

# Display a few rows from gene_annotation to understand its structure
print("\nSample rows from gene annotation:")
print(gene_annotation.head())

# Display a few rows from gene_data to understand its structure
print("\nSample rows from gene expression data:")
print(gene_data.head(3))

# Create a more direct and simplified mapping approach
print("\nCreating direct gene mapping dataframe...")
# Create a dataframe with just ID and gene identifiers (as Gene)
direct_mapping = gene_annotation[['ID', 'ENTREZ_GENE_ID']].copy()
direct_mapping = direct_mapping.rename(columns={'ENTREZ_GENE_ID': 'Gene'})
direct_mapping = direct_mapping.dropna(subset=['Gene'])  # Remove rows with missing gene identifiers
direct_mapping['Gene'] = direct_mapping['Gene'].astype(str)  # Ensure Gene column is string type

# Make sure mapping IDs match expression IDs in format
direct_mapping = direct_mapping[direct_mapping['ID'].isin(gene_data.index)]

print(f"Created direct mapping with {len(direct_mapping)} rows")
print("Preview of mapping:")
print(direct_mapping.head())

# Debug: check that some probe IDs from expression data exist in our mapping
first_10_probe_ids = list(gene_data.index[:10])
print(f"\nChecking if first 10 probe IDs from expression data exist in mapping:")
for probe_id in first_10_probe_ids:
    exists = probe_id in direct_mapping['ID'].values
    print(f"Probe ID {probe_id}: {'exists' if exists else 'does not exist'} in mapping")

# Apply mapping with the simpler approach
print("\nApplying gene mapping with direct approach...")
# Create a custom list for Gene column to ensure it's processed as a list
direct_mapping['Gene'] = direct_mapping['Gene'].apply(lambda x: [x])  # Wrap each gene ID in a list

# Apply the mapping
gene_data = apply_gene_mapping(gene_data, direct_mapping)
print(f"Created gene expression data with {len(gene_data)} genes")

# If the direct mapping fails, try using the Description field
if len(gene_data) == 0:
    print("\nDirect mapping failed. Trying approach with Description field...")
    # Extract gene symbols from Description column
    desc_mapping = gene_annotation[['ID', 'Description']].copy()
    desc_mapping = desc_mapping.dropna(subset=['Description'])
    
    # Extract the gene name from the beginning of the description
    def extract_first_gene_name(desc):
        if pd.isna(desc):
            return []
        # Take the first part of the description which often contains the gene name
        desc = str(desc).strip()
        parts = desc.split(',')[0].split('(')[0].strip()
        return [parts]  # Return as a list for compatibility with apply_gene_mapping
    
    desc_mapping['Gene'] = desc_mapping['Description'].apply(extract_first_gene_name)
    desc_mapping = desc_mapping[desc_mapping['ID'].isin(gene_data.index)]
    
    print(f"Created description-based mapping with {len(desc_mapping)} rows")
    print("Preview of description mapping:")
    print(desc_mapping.head())
    
    # Apply the description-based mapping
    gene_data = apply_gene_mapping(gene_data, desc_mapping)
    print(f"Created gene expression data with {len(gene_data)} genes")

# Print preview of the gene expression data
print("\nPreview of gene expression data:")
print(preview_df(gene_data))

# Normalize gene symbols
if len(gene_data) > 0:
    gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: gene expression data with {len(gene_data)} genes")
else:
    print("Normalization skipped as gene data is empty")

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# Update the gene availability flag based on results
is_gene_available = len(gene_data) > 0
print(f"Final status - Gene expression data available: {is_gene_available}")


Analyzing gene identifier and symbol columns...
Comparing gene expression IDs with annotation IDs...
Example ID from expression data: 100009676_at
Number of matching probe IDs: 18989 out of 18989

Sample rows from gene annotation:
         ID ENTREZ_GENE_ID                                        Description  \
0      1_at              1                             alpha-1-B glycoprotein   
1     10_at             10  N-acetyltransferase 2 (arylamine N-acetyltrans...   
2    100_at            100                                adenosine deaminase   
3   1000_at           1000          cadherin 2, type 1, N-cadherin (neuronal)   
4  10000_at          10000  v-akt murine thymoma viral oncogene homolog 3 ...   

  SPOT_ID  
0     NaN  
1     NaN  
2     NaN  
3     NaN  
4     NaN  

Sample rows from gene expression data:
              GSM2473069  GSM2473070  GSM2473071  GSM2473072  GSM2473073  \
ID                                                                         
100009676_at    2.

Created direct mapping with 1329168 rows
Preview of mapping:
         ID   Gene
0      1_at      1
1     10_at     10
2    100_at    100
3   1000_at   1000
4  10000_at  10000

Checking if first 10 probe IDs from expression data exist in mapping:
Probe ID 100009676_at: exists in mapping
Probe ID 10000_at: exists in mapping
Probe ID 10001_at: exists in mapping
Probe ID 10002_at: exists in mapping
Probe ID 10003_at: exists in mapping
Probe ID 100048912_at: exists in mapping
Probe ID 100049716_at: exists in mapping
Probe ID 10004_at: exists in mapping
Probe ID 10005_at: exists in mapping
Probe ID 10006_at: exists in mapping

Applying gene mapping with direct approach...


Created gene expression data with 0 genes

Direct mapping failed. Trying approach with Description field...
Created description-based mapping with 0 rows
Preview of description mapping:
Empty DataFrame
Columns: [ID, Description, Gene]
Index: []
Created gene expression data with 0 genes

Preview of gene expression data:
{'Description': [], 'GSM2473069': [], 'GSM2473070': [], 'GSM2473071': [], 'GSM2473072': [], 'GSM2473073': [], 'GSM2473074': [], 'GSM2473075': [], 'GSM2473076': [], 'GSM2473077': [], 'GSM2473078': [], 'GSM2473079': [], 'GSM2473080': [], 'GSM2473081': [], 'GSM2473082': [], 'GSM2473083': [], 'GSM2473084': [], 'GSM2473085': [], 'GSM2473086': [], 'GSM2473087': [], 'GSM2473088': [], 'GSM2473089': [], 'GSM2473090': [], 'GSM2473091': [], 'GSM2473092': [], 'GSM2473093': [], 'GSM2473094': [], 'GSM2473095': [], 'GSM2473096': [], 'GSM2473097': [], 'GSM2473098': [], 'GSM2473099': [], 'GSM2473100': [], 'GSM2473101': [], 'GSM2473102': [], 'GSM2473103': [], 'GSM2473104': [], 'GSM2473105

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    # Check if gene_data is empty before normalization
    if gene_data.empty:
        print("Gene data is empty. Skipping normalization.")
        normalized_gene_data = gene_data
    else:
        normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
        print(f"After normalization: {len(normalized_gene_data.index)} unique gene symbols")
    
    # Save the normalized gene data (even if empty, to maintain consistent workflow)
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene symbols: {e}")
    normalized_gene_data = gene_data  # Use original data if normalization fails

# Use is_trait_available from previous steps, don't recheck trait_row
print("\nChecking trait availability from previous analysis...")
# Load the clinical data from the previously saved file
try:
    clinical_df = pd.read_csv(out_clinical_data_file)
    print(f"Loaded clinical data with shape: {clinical_df.shape}")
    is_trait_available = True
except Exception as e:
    print(f"Error loading clinical data: {e}")
    clinical_df = pd.DataFrame()
    is_trait_available = False

# 2. Link clinical and genetic data if available
print("\nLinking clinical and genetic data...")
try:
    if is_trait_available and not normalized_gene_data.empty:
        # Print sample IDs from both datasets for debugging
        print("First few clinical sample columns:", list(clinical_df.columns)[:5])
        print("First few genetic sample columns:", list(normalized_gene_data.columns)[:5])
        
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        
        # Check if we have at least one sample with trait value
        if trait in linked_data.columns:
            trait_count = linked_data[trait].count()
            print(f"Number of samples with trait values: {trait_count}")
            
            if trait_count > 0:
                # 3. Handle missing values systematically
                print("\nHandling missing values...")
                linked_data = handle_missing_values(linked_data, trait)
                print(f"After handling missing values, data shape: {linked_data.shape}")
                
                # Check if we still have samples after missing value handling
                if linked_data.shape[0] > 0:
                    # 4. Determine whether the trait and demographic features are biased
                    print("\nChecking for bias in features...")
                    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                else:
                    print("Error: All samples were removed during missing value handling.")
                    is_biased = True
            else:
                print("No samples have valid trait values. Dataset cannot be used.")
                is_biased = True
        else:
            print(f"Trait column '{trait}' not found in linked data.")
            is_biased = True
    else:
        print("Cannot link data: clinical or genetic data is missing.")
        if not is_trait_available:
            print("Reason: Trait data is not available.")
        if normalized_gene_data.empty:
            print("Reason: Gene expression data is empty (mapping failed).")
        linked_data = pd.DataFrame()
        is_biased = True
        
except Exception as e:
    print(f"Error in linking clinical and genetic data: {e}")
    linked_data = pd.DataFrame()
    is_biased = True

# 5. Final quality validation
print("\nPerforming final validation...")
note = "Dataset failed preprocessing: gene mapping in Step 6 produced empty data. "
note += "The source data contains Affymetrix probe IDs but could not map to human gene symbols properly."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=False,  # Set to False since gene mapping failed
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data if 'linked_data' in locals() and not linked_data.empty else pd.DataFrame(),
    note=note
)

# 6. Save linked data if usable
if is_usable and 'linked_data' in locals() and not linked_data.empty:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")


Normalizing gene symbols...
Gene data is empty. Skipping normalization.
Normalized gene expression data saved to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE94321.csv

Checking trait availability from previous analysis...
Loaded clinical data with shape: (1, 5)

Linking clinical and genetic data...
Cannot link data: clinical or genetic data is missing.
Reason: Gene expression data is empty (mapping failed).

Performing final validation...
Abnormality detected in the cohort: GSE94321. Preprocessing failed.
Dataset not usable for Kidney_Clear_Cell_Carcinoma association studies. Data not saved.
